<a href="https://colab.research.google.com/github/MadsNL/AMLMain/blob/Mads/AML_IceCores_latent_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! gdown --id 1paSsYReykEVDPX8pA2SUf7Gr7Lptq-PX
! unzip Insoluables_TrainingData26May2022.zip
!git clone https://github.com/MadsNL/AMLpackages.git
import sys
sys.path.append('/content/AMLpackages/')
from load_data import training_set, not_training_set
!pip install pyyaml h5py 
!pip install umap-learn

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1paSsYReykEVDPX8pA2SUf7Gr7Lptq-PX 

unzip:  cannot find or open Insoluables_TrainingData26May2022.zip, Insoluables_TrainingData26May2022.zip.zip or Insoluables_TrainingData26May2022.zip.ZIP.
Cloning into 'AMLpackages'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 12 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.
Looking in indexes: https://pypi.org/simple, h

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import umap
import seaborn as sb
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [3]:
sup_path = '/content/Insoluables_TrainingData/train/'
sup_im_path = '/content/Insoluables_TrainingData/train/'
sup_df = pd.read_csv(sup_path+'train.csv')
train_df, test_df = train_test_split(sup_df, test_size=0.2)
train = training_set(train_df, sup_im_path)
test = training_set(test_df, sup_im_path)

FileNotFoundError: ignored

In [ ]:
# save image_size
im_e, _, _ = train.__getitem__(0)
image_size = im_e.shape[0]
image_size

In [ ]:
train_size = len(train.labels)
test_size = len(test.labels)

In [ ]:
# create encoder training
x_train = []
x_train_lab = []
x_train_feats = []
nr_ims = 5000 # 10-15k images are the max we can load with the memory we have
for i in range(nr_ims):
    im, lab, feat = train.__getitem__(i)
    x_train.append(im)
    x_train_lab.append(lab)
    x_train_feats.append(feat)
x_train = np.array(x_train)
x_train_lab = np.array(x_train_lab)
x_train_feats = np.array(x_train_feats)

# create testing
x_test = []
x_test_lab = []
x_test_feats = []
nr_ims = 5000
for i in range(nr_ims):
    im, lab, feat = test.__getitem__(i)
    x_test.append(im)
    x_test_lab.append(lab)
    x_test_feats.append(feat)
x_test = np.array(x_test)
x_test_lab = np.array(x_test_lab)
x_test_feats = np.array(x_test_feats)

In [ ]:
latent_dims = [2,16,32,64,128,128*2,128*2*2]
accs = []

for i in range(len(latent_dims)):

    latent_dim = latent_dims[i]
    image_size= 128
    unit_n = 32

    encoder_input = keras.Input(shape=(image_size,image_size,1))
    x = keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2,2), activation='relu')(encoder_input)
    x = keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2,2), activation='relu')(x)
    x = keras.layers.Flatten()(x)
    encoder_output = keras.layers.Dense(latent_dim)(x)

    encoder = keras.Model(encoder_input, encoder_output, name='encoder')

    decoder_input = keras.layers.InputLayer(input_shape=latent_dim)(encoder_output)
    x = keras.layers.Dense(units=unit_n*unit_n*32, activation=tf.nn.relu)(decoder_input)
    x = keras.layers.Reshape(target_shape=(unit_n, unit_n, 32))(x)
    x = keras.layers.Conv2DTranspose(
        filters=64, kernel_size=3, strides=2, padding='same',
        activation='relu')(x)
    x = keras.layers.Conv2DTranspose(
        filters=32, kernel_size=3, strides=2, padding='same',
        activation='relu')(x)
    # No activation
    decoder_output = keras.layers.Conv2DTranspose(
        filters=1, kernel_size=3, strides=1, padding='same')(x)

    autoencoder = keras.Model(encoder_input, decoder_output, name='autoencoder')

    opt = keras.optimizers.Adam(lr=0.001, decay=1e-6)

    encoder.compile(opt, loss='mse')
    autoencoder.compile(opt, loss='mse')

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)

    autoencoder.fit(x_train, x_train, epochs=200, batch_size=32, validation_split=0.1, callbacks=[callback])

    encoder_preds_train = encoder.predict(x_train)
    encoder_preds_test = encoder.predict(x_test)


    scalar_encoder_combined_train = []
    for i in range(len(x_train_feats)):
        comb = np.append(x_train_feats[i], encoder_preds_train[i])
        scalar_encoder_combined_train.append(comb)
    scalar_encoder_combined_train = np.array(scalar_encoder_combined_train)

    scalar_encoder_combined_test = []
    for i in range(len(x_test_feats)):
        comb = np.append(x_test_feats[i], encoder_preds_test[i])
        scalar_encoder_combined_test.append(comb)
    scalar_encoder_combined_test = np.array(scalar_encoder_combined_test)

    tree_combined = lgb.LGBMClassifier(objective='multiclass')
    tree_combined.fit(scalar_encoder_combined_train, x_train_lab)

    tree_combined_preds_test = tree_combined.predict(scalar_encoder_combined_test)

    acc_combined = sum(tree_combined_preds_test==x_test_lab)/len(x_test_lab)
    accs.append(acc_combined)

In [ ]:
print(accs)